In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Employee.csv')
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [3]:
print(df['Education'].value_counts())
print(df['City'].value_counts())
print(df['Gender'].value_counts())
print(df['EverBenched'].value_counts())

Bachelors    3601
Masters       873
PHD           179
Name: Education, dtype: int64
Bangalore    2228
Pune         1268
New Delhi    1157
Name: City, dtype: int64
Male      2778
Female    1875
Name: Gender, dtype: int64
No     4175
Yes     478
Name: EverBenched, dtype: int64


In [4]:
df['Education'] = df['Education'].apply(lambda x: 0 if x == 'Bachelors' else (1 if x == 'Masters' else 2))
df['City'] = df['City'].apply(lambda x: 0 if x == 'Bangalore' else (1 if x == 'Pune' else 2))
df['Gender'] = df['Gender'].apply(lambda x: 0 if x == 'Male' else 1)
df['EverBenched'] = df['EverBenched'].apply(lambda x: 0 if x == 'No' else 1)

In [5]:
df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,0,2017,0,3,34,0,0,0,0
1,0,2013,1,1,28,1,0,3,1
2,0,2014,2,3,38,1,0,2,0
3,1,2016,0,3,27,0,0,5,1
4,1,2017,1,3,24,0,1,2,1
...,...,...,...,...,...,...,...,...,...
4648,0,2013,0,3,26,1,0,4,0
4649,1,2013,1,2,37,0,0,2,1
4650,1,2018,2,3,27,0,0,5,1
4651,0,2012,0,3,30,0,1,2,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Education                  4653 non-null   int64
 1   JoiningYear                4653 non-null   int64
 2   City                       4653 non-null   int64
 3   PaymentTier                4653 non-null   int64
 4   Age                        4653 non-null   int64
 5   Gender                     4653 non-null   int64
 6   EverBenched                4653 non-null   int64
 7   ExperienceInCurrentDomain  4653 non-null   int64
 8   LeaveOrNot                 4653 non-null   int64
dtypes: int64(9)
memory usage: 327.3 KB


In [7]:
df.shape

(4653, 9)

In [8]:
df['LeaveOrNot'].isnull().sum()

0

In [9]:
# now dividing the dataframe into dependent and independent varaible
X=df.drop(['LeaveOrNot'], axis=1)
y=df.LeaveOrNot

# check the shape
X.shape, y.shape

((4653, 8), (4653,))

In [10]:
# we will divide the dataset into training and testing dataset
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=99)

# check the shape again
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3722, 8), (931, 8), (3722,), (931,))

In [11]:
# scaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [12]:
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [13]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1])

# check the shape again
X_train.shape, X_test.shape

((3722, 8), (931, 8))

In [14]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

Before OverSampling, counts of label '1': 1283
Before OverSampling, counts of label '0': 2439 



In [17]:
from imblearn.combine import SMOTEENN

# oversampling the train dataset using SMOTE
smt= SMOTEENN()
X_train_sm, y_train_sm= smt.fit_resample(X_train, y_train.ravel())

X_train_sm= X_train_sm.reshape(X_train_sm.shape[0],X_train_sm.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [18]:
print('After OverSampling, the shape of train_X: {}'.format(X_train_sm.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_sm.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_sm==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_sm==0)))

After OverSampling, the shape of train_X: (3087, 8, 1)
After OverSampling, the shape of train_y: (3087,) 

After OverSampling, counts of label '1': 1571
After OverSampling, counts of label '0': 1516


In [20]:
# import the libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv1D,BatchNormalization,Dropout

In [21]:
# import model
model=Sequential()
# layers
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=X_train_sm[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# build ANN
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))

In [22]:
# summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 7, 32)             96        
                                                                 
 batch_normalization (BatchN  (None, 7, 32)            128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 7, 32)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 64)             4160      
                                                                 
 batch_normalization_1 (Batc  (None, 6, 64)            256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 6, 64)             0

In [23]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
%%time
# fitting the model
history=model.fit(X_train_sm,y_train_sm,epochs=15,validation_data=(X_test,y_test))

Epoch 1/15
97/97 [==============================] - 3s 9ms/step - loss: 3.8059 - accuracy: 0.7042 - val_loss: 4.2992 - val_accuracy: 0.6649
Epoch 2/15
97/97 [==============================] - 1s 7ms/step - loss: 3.1595 - accuracy: 0.7658 - val_loss: 4.6098 - val_accuracy: 0.6563
Epoch 3/15
97/97 [==============================] - 1s 6ms/step - loss: 3.3031 - accuracy: 0.7609 - val_loss: 4.2973 - val_accuracy: 0.6778
Epoch 4/15
97/97 [==============================] - 1s 6ms/step - loss: 2.9809 - accuracy: 0.7833 - val_loss: 4.3082 - val_accuracy: 0.6799
Epoch 5/15
97/97 [==============================] - 1s 6ms/step - loss: 3.1057 - accuracy: 0.7700 - val_loss: 4.4474 - val_accuracy: 0.6864
Epoch 6/15
97/97 [==============================] - 1s 6ms/step - loss: 2.8068 - accuracy: 0.7956 - val_loss: 4.4017 - val_accuracy: 0.6907
Epoch 7/15
97/97 [==============================] - 1s 6ms/step - loss: 2.7095 - accuracy: 0.8037 - val_loss: 4.2835 - val_accuracy: 0.7111
Epoch 8/15
97/97 [==

# BFloat-16

In [25]:
# import model
model2=Sequential()
# layers
model2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=X_train_sm[0].shape))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))

model2.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))

# build ANN
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='relu'))

In [26]:
# convert model weights to bfloat16 data type
policy = tf.keras.mixed_precision.Policy('mixed_bfloat16')
tf.keras.mixed_precision.set_global_policy(policy)

In [27]:
# convert input data to bfloat16 data type
X_train_bfloat = tf.convert_to_tensor(X_train_sm, dtype=tf.bfloat16)

In [28]:
X_train_bfloat.shape

TensorShape([3087, 8, 1])

In [29]:
type(X_train_bfloat)

tensorflow.python.framework.ops.EagerTensor

In [30]:
# convert test data to bfloat16 data type
X_test_bfloat = tf.convert_to_tensor(X_test, dtype=tf.bfloat16)

In [31]:
# train the model with bfloat16 data type
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
%%time
# fitting the model
model2.fit(X_train_bfloat, y_train_sm, epochs=15,validation_data=(X_test_bfloat,y_test))

Epoch 1/15
97/97 [==============================] - 1s 6ms/step - loss: 2.4696 - accuracy: 0.8257 - val_loss: 4.4328 - val_accuracy: 0.6992
Epoch 2/15
97/97 [==============================] - 1s 5ms/step - loss: 2.4212 - accuracy: 0.8264 - val_loss: 3.9365 - val_accuracy: 0.7229
Epoch 3/15
97/97 [==============================] - 1s 5ms/step - loss: 2.7153 - accuracy: 0.8037 - val_loss: 3.9221 - val_accuracy: 0.7358
Epoch 4/15
97/97 [==============================] - 1s 5ms/step - loss: 2.5640 - accuracy: 0.8137 - val_loss: 3.7276 - val_accuracy: 0.7444
Epoch 5/15
97/97 [==============================] - 1s 6ms/step - loss: 2.4740 - accuracy: 0.8247 - val_loss: 3.9262 - val_accuracy: 0.7390
Epoch 6/15
97/97 [==============================] - 1s 5ms/step - loss: 2.3861 - accuracy: 0.8299 - val_loss: 3.9078 - val_accuracy: 0.7347
Epoch 7/15
97/97 [==============================] - 1s 6ms/step - loss: 2.2675 - accuracy: 0.8367 - val_loss: 4.2786 - val_accuracy: 0.7078
Epoch 8/15
97/97 [==